# IEnumerable / IEnumerator

## Итераторы и контейнеры ###

Что такое контейнер и что такое итератор общими словами:
* контейнер - хранилище объектов одного типа
* итератор - "ключик" к конкретному объекту в контейнере, возможно, позволяющий "обходить" контейнер - перечислять объекты в нём




## Паттерн проектирования итератор ###
![](drawings.drawio.svg)

Напишем реализацию

```c#
class Client
{
    public void Iterate()
    {
        // Сам объект, по которому мы итерируемся
        IAggregatable<int> a = new ConcreteAggregate<int>([1,2,3,4]); 

        var indexedObject = ConcreateAggregate[0];

        // Наш ключик, по которому мы можем двигаться по объекту
        var iterator = a.CreateIterator();

        // Item - содержимое, которое мы получим по итератору.
        var item = 0;
        // Пока итератор может двигаться, мы движемся.
        while (!iterator.IsDone())
        {
            item = iterator.Next(); // Выдвигаемся на следующую позицию
            Console.WriteLine(item); // 1 2 3 4
        }
    }
}

interface IAggregatable<T>
{
    public Iterator<T> CreateIterator();
    public int Count { get; }
    public T this[int index] { get; set; }
}

class ConcreteAggregate<T> : IAggregatable<T>
{
    private readonly List<T> _items = new(); // Сюда не смотрим

    public ConcreteAggregate(List<T> items)
    {
        _items = items;
    }

    public Iterator<T> CreateIterator()
    {
        return new ConcreteIterator<T>(this);
    }

    public int Count
    {
        get { return _items.Count; }
    }

    public T this[int index]
    {
        get { return _items[index]; }
        set { _items.Insert(index, value); }
    }
}

public interface Iterator<T>
{
    public abstract T Next();
    public abstract bool IsDone();
    public abstract T CurrentItem();
}

class ConcreteIterator<T> : Iterator<T>
{
    private readonly IAggregatable<T> _aggregate;
    private int _current = -1;

    public ConcreteIterator(IAggregatable<T> aggregate)
    {
        _aggregate = aggregate;
    }

    public T Next()
    {
        T ret = default(T);

        _current++;

        if (_current < _aggregate.Count)
        {
            ret = _aggregate[_current];
        }

        return ret;
    }

    public T CurrentItem()
    {
        return _aggregate[_current];
    }

    public bool IsDone()
    {
        return _current >= _aggregate.Count - 1;
    }
}
```

А че делать если в обратном порядке хотим также пройти.

Можно реализовать свой собственный обратный итератор:

```c#
public class Client
{
    ...

    public void IterateReverse()
    {
        IAggregatable<int> a = new ConcreteAggregate<int>([1, 2, 3, 4]);

        var iterator = a.CreateReverseIterator();

        int item = 0;
        while (!iterator.IsDone())
        {
            item = iterator.Next();
            Console.WriteLine(item);
        }
    }
}

interface IAggregatable<T>
{
    public Iterator<T> CreateIterator();
    public Iterator<T> CreateReverseIterator(); // Добавили в интерфейс обратный итератор
    public int Count { get; }
    public T this[int index] { get; set; }
}

class ConcreteAggregate<T> : IAggregatable<T>
{
    ...

    public Iterator<T> CreateReverseIterator()
    {
        return new ConcreteReverseIterator<T>(this);
    }

    ...
}

class ConcreteReverseIterator<T> : Iterator<T>
{
    private readonly IAggregatable<T> _aggregate;
    private int _current;

    public ConcreteReverseIterator(IAggregatable<T> aggregate)
    {
        _aggregate = aggregate;
        _current = _aggregate.Count;
    }

    public T Next()
    {
        T ret = default(T);

        _current--;

        if (_current != -1)
        {
            ret = _aggregate[_current];
        }

        return ret;
    }

    public T CurrentItem()
    {
        return _aggregate[_current];
    }

    public bool IsDone()
    {
        return _current == 0;
    }
}
```

***Вопрос*** В чем проблема данного подхода?

<details>
<summary>Ответ</summary>

1) Чтобы пройтись по контейнеру нужно написать много кода.

    var iterator = a.CreateReverseIterator();

    int item = 0;

    while (!iterator.IsDone())

    {

        item = iterator.Next()

    }

2) Нужно в этом всем не запутаться. Простой обход по индексу намного проще воспринимается

</details>

## Что придумали 

```c#
public interface IEnumerable<out T> : IEnumerable
{
    new IEnumerator<T> GetEnumerator();
}

public interface IEnumerable
{
    IEnumerator GetEnumerator();
}

public interface IEnumerator<out T> : IDisposable, IEnumerator
{    
    new T Current {
        get; 
    }
}

public interface IEnumerator
{
    bool MoveNext();

    Object Current {
        get; 
    }

    void Reset();
}

```

***Вопросы***

## Реализуем обход используя данные интерфейсы

```c#
public class MyList<T> : IEnumerable<T>
{
    private T[] _elements;

    public MyList(T[] elements)
    {
        _elements = elements;
    }

    public IEnumerator<T> GetEnumerator() => new MyListEnumerator<T>(_elements);
    IEnumerator IEnumerable.GetEnumerator() => GetEnumerator();
}

public class MyListEnumerator<T> : IEnumerator<T>
{
    private readonly T[] _elements;

    public MyListEnumerator(T[] elements)
    {
        _elements = elements;
    }

    int currentIndex = -1;
    public T Current => _elements[currentIndex];
    object IEnumerator.Current => throw new NotImplementedException();

    public void Dispose() { }

    public bool MoveNext()
    {
        currentIndex++;

        if (currentIndex < _elements.Length)
        {
            return true;
        }

        return false;
    }

    public void Reset() => currentIndex = 0;
}
```

### ??? И что поменялось ??? ###

```c#
var list = new MyList<int>([0, 1, 2, 3]);

foreach (var element in list)
{
    //Do something
}
```

### Неожиданно

```c#
public class MyList<T> //: IEnumerable<T>
{
    private T[] _elements;

    public MyList(T[] elements)
    {
        _elements = elements;
    }
    
    public IEnumerator<T> GetEnumerator() => new MyListEnumerator<T>(_elements);
    //IEnumerator IEnumerable.GetEnumerator() => GetEnumerator();
}

...

var list = new MyList<int>([0, 1, 2, 3]);

foreach (var element in list) // still working
{
    //Do something
}
```

### Сделаем красиво 
```c#
public class MyList<T> : IEnumerable<T>
{

    public MyListEnumerator<T> GetEnumerator() => new MyListEnumerator<T>(_elements); // Поменяли IEnumerator<T> на MyListEnumerator<T>
                                                                                      // Теперь не боксится
    IEnumerator IEnumerable.GetEnumerator() => GetEnumerator();                       // Уже было
    IEnumerator<T> IEnumerable<T>.GetEnumerator() => GetEnumerator();                 // Добавили явный полиморфный вариант
}
```

### Смешной пример 
```c#
var list = new MyList<int>([1, 2, 3]);

var polyIterator = new { Items = ((IEnumerable<int>) list).GetEnumerator() }; 
while (polyIterator.Items.MoveNext())
{
    Console.WriteLine(polyIterator.Items.Current);
}

Console.ReadLine();

var duckIterator = new { Items = list.GetEnumerator() };
while (duckIterator.Items.MoveNext())
{
    Console.WriteLine(duckIterator.Items.Current);
}
```

***Вопрос*** Что выведется на экране
* 1 2 3 1 2 3
* 1<br/> 2<br/> 3<br/> 1<br/> 2<br/> 3<br/>
* 1<br/> 2<br/> 3<br/>


* OutOfRangeException
* BDS

## Генераторы

Тут все как в питоне
```C#
static IEnumerable<int> Fibonacci(int count)
{
    int stopValue = 10;
    int previous = 1;
    int current = 1;

    for (int i = 0; i < count; i++)
    {
        if (i >= stopValue)
        {
            yield break;
        }

        yield return previous;
        int temp = previous + current;
        previous = current;
        current = temp;
    }
}

...

foreach (var number in Fibonacci(5))
{
    Console.Write(number + " "); // 1 1 2 3 5 (5 чисел)
}

foreach (var number in Fibonacci(150))
{
    Console.Write(number + " "); // 1 1 2 3 5 8 13 21 34 55 (10 чисел)
}
```

Можно и внутри генератора другой генератор использовать.

```c#
static IEnumerable<int> Fibonacci(int count)
{
    int stopValue = 10;
    int previous = 1;
    int current = 1;

    for (int i = 0; i < count; i++)
    {
        if (i > stopValue)
        {
            yield break;
        }
        
        foreach (var num in Fibonacci(i)) // Внутри вызываем еще один генератор
        {
            yield return num;
        }

        yield return previous;
        int temp = previous + current;
        previous = current;
        current = temp;
    }
}
```

## Как работает внутри

На самом деле тут очень много кодогенерации, и все это преобразутеся <br/>
в известные нам IEnumerable/IEnumerator.

Тут у нас все как с обычным foreach, также получаем итератор, и после проходимся через MoveNext()

```c#
IEnumerator<int> enumerator = Fibonacci(20).GetEnumerator();
try
{
    while (enumerator.MoveNext())
    {
        Console.Write(string.Concat(enumerator.Current.ToString(), " "));
    }
}
finally
{
    if (enumerator != null)
    {
        enumerator.Dispose();
    }
}
```

Вот сама функция, которая предоставляет на итератор.

```c#
private static IEnumerable<int> Fibonacci(int count)
{
    var enumerator = new FibonacciEnumerator(-2); // Что за -2
    enumerator.Counter = count; // Тут мы выставляем наше количество чисел
    return enumerator;
}
```

Вот состояния в конкретном случае (возможно это не правильные обозначения)
* -m   Поврежден на внутреннем блоке
* ...
* -2   Не ициализирована
* -1   Поврежден
*  0   Инициализирован
*  1   Проходит по циклу
* ...
*  n   Следующие блоки итератора

### Инициализация итератора

```c#
IEnumerator<int> IEnumerable<int>.GetEnumerator()
{
    FibonacciEnumerator enumerator;

    if (EnumeratorState == -2 _currentThreadId == Environment.CurrentManagedThreadId)
    {
        EnumeratorState = 0;
        enumerator = this;
    }
    else
    {
        enumerator = new FibonacciEnumerator(0);
    }

    enumerator.CaptureCount = Counter;
    return this;
}
```

### Самое интересное

```c#
private bool MoveNext()
{
    int num = EnumeratorState;

    if (num != 0)
    {
        if (num != 1)
        {
            return false;
        }

        EnumeratorState = -1;
        _temp = _previous + _current;
        _previous = _current;
        _current = _temp;
        _i++;
    }
    else
    {   
                              // Сюда мы заходим при первом MoveNext()
                              // Больше мы сюда не заходим.
        EnumeratorState = -1; // Выставляем сюда -1, если не дойдем до
                              // EnumeratorState = 1, значит у нас произошла ошибка.
        _stopValue = 10;
        _stopValue = 10;
        _previous = 1;
        _current = 1;
        _i = 0;
    }
    if (_i < CaptureCount)
    {
        if (_i > _stopValue)
        {
            return false;
        }

        CurrentIteratorValue = previous;
        EnumeratorState = 1;

        return true;
    }

    return false;
}
```


Это был простой пример, но если блоков много, то он может это все в свитчкейсы засунуть. При малом количестве <br/>
операторов yield он это генерирует через if else, и переодически пользуется goto. <br/>
Если кому-то и в правду интересно смотреть на это, можете просто скопипастить пример на https://sharplab.io/